### Importing...

In [1]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
import seaborn as sns
import pandas as pd
import numpy as np
# import zipfile
import os
%matplotlib inline

### Data Collection


In [2]:
# !mkdir ~/.kaggle
# !cp "/content/kaggle.json" ~/.kaggle/kaggle.json 
# os.environ["KAGGLE_CONFIG_"] ='/content/kaggle.json'
# !kaggle competitions download -c kernel-methods-ammi-2022

  0% 0.00/464k [00:00<?, ?B/s]
100% 464k/464k [00:00<00:00, 98.0MB/s]


In [4]:
# !unzip "/content/kernel-methods-ammi-2022.zip" -d data/

Archive:  /content/kernel-methods-ammi-2022.zip
  inflating: data/Xte.csv            
  inflating: data/Xte_vectors.csv    
  inflating: data/Xtr.csv            
  inflating: data/Xtr_vectors.csv    
  inflating: data/Ytr.csv            


### Loading Data

In [2]:
x=pd.read_csv('data/Xtr_vectors.csv')
print(x.shape)
x.head()

(2000, 65)


,Id,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,1,0.1406,0.0625,0.0781,0.1094,0.0625,0.0312,0.0312,0.1406,0.0781,...,0.0625,0.1719,0.0781,0.0312,0.0625,0.1719,0.0781,0.1094,0.1406,0.1250
1,2,0.1250,0.1250,0.0781,0.0938,0.0625,0.0469,0.0625,0.1094,0.1406,...,0.0781,0.1406,0.1094,0.0625,0.0312,0.1406,0.0625,0.1406,0.1250,0.1406
2,3,0.1406,0.1719,0.0156,0.0781,0.1406,0.1094,0.0938,0.0469,0.0312,...,0.1406,0.0938,0.0938,0.0156,0.0625,0.0781,0.1562,0.0469,0.0781,0.1094
3,4,0.1406,0.0938,0.0781,0.1250,0.1250,0.0469,0.0312,0.0781,0.0938,...,0.0312,0.2031,0.1094,0.0469,0.0469,0.1875,0.1094,0.1562,0.1406,0.1875
4,5,0.2188,0.1094,0.1094,0.2031,0.1250,0.0625,0.0156,0.1094,0.1406,...,0.0312,0.0938,0.0781,0.0156,0.0312,0.1250,0.1719,0.0781,0.0625,0.1719


In [3]:
sub=pd.read_csv("data/Xte_vectors.csv")
print(sub.shape)

(1000, 65)


In [4]:
y=pd.read_csv("data/Ytr.csv")
y.head(2)

,Id,Covid
0,1,1
1,2,1


In [5]:
print(x.shape,y.shape,sub.shape)

(2000, 65) (2000, 2) (1000, 65)


In [6]:
x.drop(['Id'],axis=1,inplace=True)
y.drop(['Id'],axis=1,inplace=True)
y.head()

,Covid
0,1
1,1
2,1
3,1
4,1


In [7]:
y['Covid'].replace(0,-1,inplace=True)
y.head()

,Covid
0,1
1,1
2,1
3,1
4,1


### Data Exploration

Checks for data decription and highly correlated columnns.

In [8]:
x.describe()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.119515,0.088961,0.079477,0.088803,0.089882,0.070451,0.075208,0.082049,0.075195,0.049875,...,0.082018,0.073139,0.066367,0.050131,0.072039,0.092185,0.097124,0.089907,0.089204,0.123997
std,0.084337,0.044590,0.041111,0.054201,0.051151,0.040540,0.040896,0.038320,0.040508,0.040903,...,0.040792,0.039112,0.034777,0.034963,0.035903,0.052865,0.055079,0.047345,0.042415,0.084406
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.062500,0.062500,0.046900,0.046900,0.046900,0.046900,0.046900,0.062500,0.046900,0.015600,...,0.046900,0.046900,0.046900,0.031200,0.046900,0.046900,0.046900,0.062500,0.062500,0.062500
50%,0.109400,0.078100,0.078100,0.093800,0.078100,0.062500,0.078100,0.078100,0.078100,0.046900,...,0.078100,0.062500,0.062500,0.046900,0.062500,0.093800,0.093800,0.093800,0.093800,0.109400
75%,0.171900,0.109400,0.109400,0.125000,0.125000,0.093800,0.093800,0.109400,0.093800,0.078100,...,0.109400,0.093800,0.093800,0.062500,0.093800,0.125000,0.140600,0.125000,0.109400,0.171900
max,0.578100,0.265600,0.250000,0.312500,0.281200,0.281200,0.265600,0.234400,0.281200,0.218800,...,0.296900,0.234400,0.187500,0.218800,0.250000,0.312500,0.312500,0.296900,0.250000,0.562500


The different features are roughly on the same scale, so we may not standardize.

In [9]:
corr = np.abs(x.corr())
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
drop_ = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
drop_

['38', '39', '42']

In [10]:
x.drop(drop_,axis=1,inplace=True)
x.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0.1406,0.0625,0.0781,0.1094,0.0625,0.0312,0.0312,0.1406,0.0781,0.0312,...,0.0625,0.1719,0.0781,0.0312,0.0625,0.1719,0.0781,0.1094,0.1406,0.1250
1,0.1250,0.1250,0.0781,0.0938,0.0625,0.0469,0.0625,0.1094,0.1406,0.0156,...,0.0781,0.1406,0.1094,0.0625,0.0312,0.1406,0.0625,0.1406,0.1250,0.1406
2,0.1406,0.1719,0.0156,0.0781,0.1406,0.1094,0.0938,0.0469,0.0312,0.0000,...,0.1406,0.0938,0.0938,0.0156,0.0625,0.0781,0.1562,0.0469,0.0781,0.1094
3,0.1406,0.0938,0.0781,0.1250,0.1250,0.0469,0.0312,0.0781,0.0938,0.0312,...,0.0312,0.2031,0.1094,0.0469,0.0469,0.1875,0.1094,0.1562,0.1406,0.1875
4,0.2188,0.1094,0.1094,0.2031,0.1250,0.0625,0.0156,0.1094,0.1406,0.0469,...,0.0312,0.0938,0.0781,0.0156,0.0312,0.1250,0.1719,0.0781,0.0625,0.1719


In [11]:
dataset=pd.concat([x,y["Covid"]],axis=1)
dataset.head()
print(x.shape,dataset.shape)

(2000, 61) (2000, 62)


### Kernel LR

In [12]:
# Prediction error
def error(ypred, ytrue):
    e = (ypred != ytrue).mean()
    return e

def add_column_ones(X):
    n = X.shape[0]
    return np.hstack([X, np.ones((n, 1))])


In [13]:
def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the RBF kernel with parameter sigma
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    sigma: float
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis = -1)
    X1_norm = np.sum(X1 ** 2, axis = -1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

In [14]:
def linear_kernel(X1, X2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the linear kernel
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix

    '''
    linear=np.dot(X1,X2.T)

    return linear

def polynomial_kernel(X1, X2, degree=2):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    where K is the polynomial kernel of degree `degree`
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    polynomial=(linear_kernel(X1, X2)+1)**degree
    return polynomial

def sigmoid(self,x):
    sig = 1/(1+np.exp(-x))
    return sig

In [15]:
class KernelMethodBase(object):
    '''
    Base class for kernel methods models
    
    Methods
    ----
    fit
    predict
    fit_K
    predict_K
    '''
    kernels_ = {
        'linear': linear_kernel,
        'polynomial': polynomial_kernel,
        'rbf': rbf_kernel#,
        # 'mismatch': mismatch_kernel,
    }
    def __init__(self, kernel='linear', **kwargs):
        self.kernel_name = kernel
        self.kernel_function_ = self.kernels_[kernel]
        self.kernel_parameters = self.get_kernel_parameters(**kwargs)
        self.fit_intercept_ = False
        
    def get_kernel_parameters(self, **kwargs):
        params = {}
        if self.kernel_name == 'rbf':
            params['sigma'] = kwargs.get('sigma', 1.)
        if self.kernel_name == 'polynomial':
            params['degree'] = kwargs.get('degree', 2)
        return params

    def fit_K(self, K, y, **kwargs):
        pass
        
    def decision_function_K(self, K):
        pass
    
    def fit(self, X, y, fit_intercept=False, **kwargs):

        if fit_intercept:
            X = add_column_ones(X)
            self.fit_intercept_ = True
        self.X_train = X
        self.y_train = y

        K = self.kernel_function_(self.X_train, self.X_train, **self.kernel_parameters)

        return self.fit_K(K, y, **kwargs)
    
    def decision_function(self, X):

        if self.fit_intercept_:
            X = add_column_ones(X)

        K_x = self.kernel_function_(X, self.X_train, **self.kernel_parameters)

        return self.decision_function_K(K_x)

    def predict(self, X):
        pass
    
    def predict_K(self, K):
        pass

In [16]:
class KernelRidgeRegression(KernelMethodBase):
    '''
    Kernel Ridge Regression
    '''
    def __init__(self, lambd=0.1, **kwargs):
        self.lambd = lambd
        # Python 3: replace the following line by
        # super().__init__(**kwargs)
        super(KernelRidgeRegression, self).__init__(**kwargs)

    def fit_K(self, K, y, sample_weights=None):
        n = K.shape[0]
        assert (n == len(y))
        
        w_sqrt = np.ones_like(y) if sample_weights is None else sample_weights
        w_sqrt = np.sqrt(w_sqrt)

        # Rescale kernel matrix K to take weights into account
        A = K * w_sqrt * w_sqrt[:, None]
        # Add lambda to the diagonal of A (= add lambda*I):
        A += n * lambd * np.eye(n)
        # self.alpha = (K + n * lambda I)^-1 y
        self.alpha = w_sqrt * np.linalg.solve(A , y * w_sqrt)

        return self
    
    def decision_function_K(self, K_x):
        return np.dot(K_x, self.alpha)
    
    def predict(self, X):
        return self.decision_function(X)
    
    def predict_K(self, K_x):
        return self.decision_function_K(K_x)

### Training

In [17]:

fold_score=[]
sub_fold_pred=[]

X=dataset
n_folds=5
n=X.shape[0]
X_train=X.drop(["Covid"],axis=1)
sub.drop(drop_,axis=1,inplace=True)

In [18]:
Y_train=y
fold_size=len(X)//n_folds
print(len(X),fold_size)

2000 400


In [19]:
sub.head(2)

,Id,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,1,0.0000,0.0312,0.1094,0.0156,0.0156,0.1250,0.0781,0.0000,0.0312,...,0.0625,0.0312,0.0156,0.1094,0.0781,0.0469,0.0312,0.0156,0.0312,0.0625
1,2,0.1562,0.1562,0.0469,0.0469,0.0938,0.1406,0.1250,0.0625,0.1094,...,0.1094,0.1250,0.0469,0.0000,0.0781,0.1094,0.1094,0.1562,0.1250,0.1406


In [20]:
kernel = 'rbf'
lambd = 0.001
sigma = 0.1
np.random.seed(2022)


model = KernelRidgeRegression(
    kernel=kernel,
    lambd=lambd,
    sigma=sigma
    )
sample_weights = np.random.rand(len(X_train)-400)


In [21]:
for folds in range(n_folds):
  indices=np.arange(folds*fold_size,(1+folds)*fold_size, dtype=int)
  
  x_val=X_train.iloc[indices]
  y_val=Y_train.iloc[indices]
  x_train=X_train[~X_train.apply(tuple,1).isin(x_val.apply(tuple,1))]
  y_train=Y_train.iloc[x_train.index]

  sample_weights = np.random.rand(x_train.shape[0])
  model.fit(x_train.to_numpy(), y_train.to_numpy(), sample_weights=sample_weights)
  ypred = np.sign(np.mean(model.predict(x_val.to_numpy()),axis=1))
  fold_score.append(f1_score(y_val,ypred))
  sub_pred=model.predict(sub.iloc[:,1:].to_numpy())
  sub_fold_pred.append(sub_pred)
np.mean(fold_score)

0.9175811118592483

In [22]:
sub_fold_pred

[array([[-0.0553748 , -0.06111947, -0.087716  , ..., -0.08004854,
         -0.07085044, -0.03674904],
        [ 0.16542282,  0.18258405,  0.26203669, ...,  0.23913145,
          0.21165367,  0.10978152],
        [ 0.64822924,  0.71547755,  1.02682231, ...,  0.93706536,
          0.82939038,  0.43019208],
        ...,
        [-0.1267854 , -0.13993831, -0.20083339, ..., -0.18327807,
         -0.16221822, -0.0841401 ],
        [-0.19260663, -0.21258794, -0.30509699, ..., -0.27842774,
         -0.24643456, -0.12782183],
        [-0.15051092, -0.16612516, -0.23841561, ..., -0.21757515,
         -0.19257433, -0.09988535]]),
 array([[-0.1143681 , -0.06377052, -0.10212303, ..., -0.005706  ,
         -0.08677218, -0.0980983 ],
        [ 0.24642217,  0.13740257,  0.22003844, ...,  0.01229437,
          0.18696289,  0.2113666 ],
        [ 1.00371561,  0.55966193,  0.8962506 , ...,  0.05007688,
          0.76152877,  0.86092885],
        ...,
        [-0.2103422 , -0.11728474, -0.18782145, ..., -

In [23]:
(sub.iloc[:,1:])

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0.0000,0.0312,0.1094,0.0156,0.0156,0.1250,0.0781,0.0000,0.0312,0.1250,...,0.0625,0.0312,0.0156,0.1094,0.0781,0.0469,0.0312,0.0156,0.0312,0.0625
1,0.1562,0.1562,0.0469,0.0469,0.0938,0.1406,0.1250,0.0625,0.1094,0.0000,...,0.1094,0.1250,0.0469,0.0000,0.0781,0.1094,0.1094,0.1562,0.1250,0.1406
2,0.0625,0.0625,0.1094,0.1250,0.0781,0.0938,0.0781,0.0625,0.1250,0.0156,...,0.0469,0.1250,0.1094,0.0156,0.0312,0.1250,0.1719,0.0781,0.1094,0.2031
3,0.1250,0.0156,0.1562,0.0156,0.0625,0.0312,0.0469,0.0938,0.0938,0.1406,...,0.0625,0.0469,0.0156,0.1250,0.0938,0.0312,0.0781,0.0938,0.0625,0.0625
4,0.0938,0.0625,0.1406,0.1250,0.1406,0.0781,0.0625,0.1250,0.1406,0.0156,...,0.0312,0.0625,0.0781,0.0469,0.0156,0.0625,0.2031,0.0625,0.0781,0.1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.1562,0.1406,0.0625,0.0781,0.0625,0.0156,0.1094,0.0938,0.1094,0.0469,...,0.0625,0.0312,0.0781,0.0625,0.1562,0.0625,0.0625,0.1094,0.0625,0.0938
996,0.0781,0.1094,0.1094,0.1562,0.1562,0.0312,0.0781,0.0312,0.0781,0.0156,...,0.0938,0.0625,0.0625,0.0312,0.0938,0.1562,0.1875,0.0781,0.1406,0.0469
997,0.0781,0.0625,0.0781,0.0469,0.0312,0.0938,0.0312,0.0938,0.0312,0.1406,...,0.0469,0.0781,0.0469,0.0781,0.1250,0.0469,0.0469,0.0781,0.1250,0.0312
998,0.0781,0.0781,0.0938,0.0312,0.0469,0.0938,0.0469,0.0625,0.0938,0.0938,...,0.1406,0.0625,0.0312,0.0781,0.0312,0.0469,0.0469,0.0156,0.0312,0.0469


### Prediction


In [24]:
submit=np.sign(np.mean(np.mean(sub_fold_pred,axis=0),axis=1)).astype(int) #we take the mean of the predictions from each fold. Then take the sign of it
print(submit.shape)

(1000,)


In [25]:
pd.DataFrame(submit).value_counts()

 1    556
-1    444
dtype: int64

### Submission

In [27]:
sub_pred=pd.DataFrame(submit,columns=['Covid'])
ss_pred=pd.concat([sub['Id'],sub_pred],axis=1)
ss_pred.tail()

,Id,Covid
995,996,1
996,997,1
997,998,-1
998,999,-1
999,1000,-1


In [28]:
ss_pred['Covid'].value_counts()

 1    556
-1    444
Name: Covid, dtype: int64

In [29]:
ss_pred['Covid'].replace(-1,0,inplace=True)


In [30]:
ss_pred["Covid"].value_counts()

1    556
0    444
Name: Covid, dtype: int64

In [27]:
sample_sub=ss_pred.to_csv('8_sub.csv',index=False)
pd.read_csv('/content/8_sub.csv')

,Id,Covid
0,1,0
1,2,1
2,3,1
3,4,0
4,5,1
...,...,...
995,996,1
996,997,1
997,998,0
998,999,0


In [ ]:
# !kaggle competitions submit -c kernel-methods-ammi-2022 -f 8_sub.csv -m "8th submission"

100% 5.76k/5.76k [00:01<00:00, 5.05kB/s]
Successfully submitted to Kernel Methods AMMI 2022

#### Sanity check with scikit-learn
***

In [55]:
from sklearn.kernel_ridge import KernelRidge

# scikit-learn conventions
gamma = 1 / (2 * sigma**2) # rbf parameter
alpha = lambd * len(y_train) # regularization

model_sklearn = KernelRidge(
        kernel=kernel,
        alpha=alpha,
        gamma=gamma
    ).fit(x_train.to_numpy(), y_train, sample_weight=sample_weights)
ypred = model_sklearn.predict(x_val.to_numpy())

# plt.figure(figsize=(8, 6))
# plt.scatter(xtrain, ytrain)
# plt.plot(xtest, ypred)
# plt.xlabel('$x$')
# plt.ylabel('$y$')
# plt.title('Kernel Ridge Regression, kernel={}, $\lambda=${}'.format(kernel, lambd))
# plt.show()

print('Difference between sklearn and ours: {:.1e}'.format(((model_sklearn.dual_coef_ - model.alpha)**2).sum()))

Difference between sklearn and ours: 8.8e+04
